In [1]:
from __future__ import print_function
import numpy as np
import deeplift
import deeplift.conversion.keras_conversion as kc

In [2]:
keras_model_weights="/srv/scratch/manyu/train/K562/ZBTB33_Seq_Meth_newModel/logdir_seq_only/model.weights.h5"
keras_model_json="/srv/scratch/manyu/train/K562/ZBTB33_Seq_Meth_newModel/logdir_seq_only/model.arch.json"
keras_model = kc.load_keras_model(weights=keras_model_weights,
                                  json=keras_model_json)


Using TensorFlow backend.


In [3]:
from deeplift.blobs import NonlinearMxtsMode
from collections import OrderedDict
method_to_model = OrderedDict()

In [10]:
method_to_model = OrderedDict()
for method_name, nonlinear_mxts_mode in [
    #The genomics default = rescale on conv layers, revealcance on fully-connected
    ('rescale_conv_revealcancel_fc', NonlinearMxtsMode.DeepLIFT_GenomicsDefault),
    ('rescale_all_layers', NonlinearMxtsMode.Rescale),
    ('revealcancel_all_layers', NonlinearMxtsMode.RevealCancel),
    ('grad_times_inp', NonlinearMxtsMode.Gradient),
    ('guided_backprop', NonlinearMxtsMode.GuidedBackprop)]:
    method_to_model[method_name] = kc.convert_functional_model(
        model=keras_model,
        nonlinear_mxts_mode=nonlinear_mxts_mode)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer activation_1 the preceding linear layer is convolution1d_1 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer activation_2 the preceding linear layer is convolution1d_2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer activation_3 the preceding linear layer is convolution1d_3 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: I assume sigmoid is the output layer, not an intermediate one; if it's an intermediate layer then please bug me and I will implement the grad func
For layer activation_4 the preceding linear layer is dense_1 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
nonl

In [7]:
deeplift_model=method_to_model['guided_backprop']
deeplift_model.get_name_to_blob().keys()

[u'activation_4',
 u'dense_1',
 u'flatten_1',
 u'averagepooling1d_1',
 u'activation_3',
 u'convolution1d_3',
 u'activation_2',
 u'convolution1d_2',
 u'activation_1',
 u'convolution1d_1',
 u'permute_1',
 u'data/genome_data_dir']

In [11]:
print("Compiling scoring functions")
method_to_scoring_func = OrderedDict()
for method, model in method_to_model.items():
    print("Compiling scoring function for: " + method)
    method_to_scoring_func[method] = model.get_target_contribs_func(
            find_scores_layer_name="data/genome_data_dir",
            pre_activation_target_layer_name="dense_1")

Compiling scoring functions
Compiling scoring function for: rescale_conv_revealcancel_fc
Compiling scoring function for: rescale_all_layers
Compiling scoring function for: revealcancel_all_layers
Compiling scoring function for: grad_times_inp
Compiling scoring function for: guided_backprop


In [28]:
# To get a function that just gives the gradients, we use the multipliers of the Gradient model
gradient_func = method_to_model['grad_times_inp'].get_target_multipliers_func(
    find_scores_layer_name="data/genome_data_dir",
    pre_activation_target_layer_name="dense_1")
print("Compiling integrated gradients scoring functions")
integrated_gradients10_func = deeplift.util.get_integrated_gradients_function(
    gradient_computation_function=gradient_func,
    num_intervals=10)
method_to_scoring_func['integrated_gradients10'] = integrated_gradients10_func

Compiling integrated gradients scoring functions


In [31]:
model_to_test = method_to_model['rescale_conv_revealcancel_fc']
deeplift_prediction_func = compile_func(
    [model_to_test.get_name_to_blob()[model_to_test.get_name_to_blob().keys()[-1]].get_activation_vars()],
    model_to_test.get_name_to_blob()[model_to_test.get_name_to_blob().keys()[0]].get_activation_vars())
original_model_predictions = keras_model.predict(onehot_data, batch_size=200)
converted_model_predictions = deeplift.util.run_function_in_batches(
    input_data_list=[data],
    func=deeplift_prediction_func,
    batch_size=200,
    progress_update=None)
print("maximum difference in predictions:",
      np.max(np.abs(np.array(converted_model_predictions) - np.array(original_model_predictions))))
assert np.max(np.array(converted_model_predictions) - np.array(original_model_predictions)) < 10 ** -5

NameError: name 'onehot_data' is not defined